In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter
import os

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
LR = 1e-3
gym.envs.register(
    id='MyCartPole-v0',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    tags={'wrapper_config.TimeLimit.max_episode_steps' : 2000},
)
env = gym.make("MyCartPole-v0")
env.reset()
goal_steps = 2000
score_requirement = 110
initial_games = 100000

[2017-07-27 00:23:31,074] Making new env: MyCartPole-v0


In [3]:
def some_random_games():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if(done):
                break

In [4]:
#some_random_games()

In [5]:
def inital_population():
    train_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            #action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if len(prev_observation) > 0:
                game_memory.append([prev_observation,action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                train_data.append([data[0], output])
        env.reset()
        scores.append(score)
    train_data_save = np.array(train_data)
    np.save('saved.npy', train_data_save)
    
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return train_data
    

In [6]:
#inital_population()

In [7]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 1024, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    
    model = tflearn.DNN(network, tensorboard_dir='log')
    
    return model

In [8]:
def train_model(train_data, model=False):
    x = np.array([i[0] for i in train_data]).reshape(-1,len(train_data[0][0]), 1)
    y = [i[1] for i in train_data]
    
    if not model:
        model = neural_network_model(input_size= len(x[0]))
        
    model.fit({'input':x}, {'targets':y}, n_epoch=3, snapshot_step=500, show_metric=True,run_id='openaistuff')
    
    return model

In [9]:
def play_game(visible=True):
    scores = []
    choices = []

    for each_game in range(100):
        score = 0
        game_memory = []
        prev_obs = []
        env.reset()
        for _ in range(goal_steps):
            if visible:
                env.render()
            if len(prev_obs) == 0:
                action = random.randrange(0,2)
            else:
                action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
            choices.append(action)

            new_obs, reward, done, info = env.step(action)
            prev_obs = new_obs
            game_memory.append([new_obs, action])
            score += reward
            if done:
                break
        scores.append(score)
    average_score = sum(scores)/len(scores)
    if visible:
        print('Scores: ',scores)
        print('Average Score ', average_score)
        print('Choice 0: {}, Choice 1: {}'.format(choices.count(0)/len(choices), choices.count(1)/len(choices)))
    return average_score


In [10]:
#training_data = inital_population()
#model = train_model(training_data)
#average_score = play_game(False)
#while average_score < 300:
#    training_data = inital_population()
#    model = train_model(training_data, model)
#    average_score = play_game(False)
#play_game()

In [11]:
training_data = inital_population()
model = train_model(training_data)
modelscore = play_game()
modelscore

Training Step: 146  | total loss: 0.65725 | time: 2.890s
| Adam | epoch: 003 | loss: 0.65725 - acc: 0.6394 -- iter: 3072/3099
Training Step: 147  | total loss: 0.65724 | time: 2.964s
| Adam | epoch: 003 | loss: 0.65724 - acc: 0.6364 -- iter: 3099/3099
--
Scores:  [644.0, 372.0, 2000.0, 214.0, 378.0, 388.0, 568.0, 694.0, 877.0, 578.0, 213.0, 594.0, 625.0, 416.0, 715.0, 188.0, 836.0, 2000.0, 2000.0, 385.0, 575.0, 204.0, 209.0, 323.0, 577.0, 662.0, 368.0, 809.0, 840.0, 2000.0, 967.0, 344.0, 245.0, 562.0, 458.0, 383.0, 521.0, 2000.0, 716.0, 2000.0, 409.0, 621.0, 2000.0, 1189.0, 2000.0, 750.0, 408.0, 614.0, 720.0, 2000.0, 203.0, 803.0, 618.0, 2000.0, 2000.0, 414.0, 226.0, 477.0, 622.0, 635.0, 316.0, 772.0, 521.0, 206.0, 427.0, 832.0, 2000.0, 821.0, 2000.0, 526.0, 535.0, 167.0, 233.0, 524.0, 610.0, 961.0, 723.0, 2000.0, 210.0, 820.0, 686.0, 635.0, 774.0, 598.0, 310.0, 2000.0, 448.0, 358.0, 732.0, 625.0, 286.0, 280.0, 582.0, 589.0, 346.0, 646.0, 371.0, 931.0, 414.0, 190.0]
Average Score  751.

751.62

In [13]:
modelscore = int(round(modelscore))
if(modelscore > 200):
    path = "models/" + str(modelscore) + "/"
    if not os.path.exists(path):
        os.makedirs(path)
    model.save(path + str(modelscore) + ".model")

INFO:tensorflow:/home/jcimbal/Dokumente/Neuronale-Netze/gym/CartPol/models/752/752.model is not in all_model_checkpoint_paths. Manually adding it.


[2017-07-27 00:30:25,717] /home/jcimbal/Dokumente/Neuronale-Netze/gym/CartPol/models/752/752.model is not in all_model_checkpoint_paths. Manually adding it.
